In [ ]:
# 031923
# YIKAI MAO

# adaptive training test (demo_2)

In [ ]:
%env PYTHONHASHSEED=0

# standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit import QuantumRegister, ClassicalRegister
from qiskit.tools.jupyter import *
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import *
#from ibm_quantum_widgets import * # does not work when not in ibm cloud platform
from qiskit.providers.aer import QasmSimulator, AerSimulator
from qiskit.transpiler.passes import RemoveBarriers
from qiskit_aer.noise import NoiseModel

# other libraries
import sys
import os
import fnmatch
import pickle
import mapomatic as mm
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from datetime import datetime
from PIL import Image

# loading your IBM Quantum account(s)
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()
provider.backends()

In [ ]:
def run_ideal_simulation(circuit, shots, seed):
    # assume perfect device with no noise

    simulator = Aer.get_backend('aer_simulator')
    temp_qc = transpile(circuit, simulator, optimization_level=0)
    job_temp = simulator.run(temp_qc, shots=shots, seed_simulator=seed)
    temp_results = job_temp.result()

    # expected output is plotted
    #plot = plot_histogram(temp_results.get_counts())
    #plot.patch.set_facecolor('xkcd:white')
    #display(plot)
    #print('raw outputs:')
    #print(temp_results.get_counts())

    # build simulation result dict
    num_clbits = circuit.num_clbits
    ideal_result_dict = {}

    for i in range(pow(2, num_clbits)):
        bin_str = format(i, str('0>' + str(num_clbits) + 'b'))
        if bin_str not in temp_results.get_counts():
            #print(bin_str + ': probability = 0')
            ideal_result_dict[bin_str] = 0
        else:
            ideal_result_dict[bin_str] = temp_results.get_counts()[bin_str]

    #print('simulation result dictionary:')
    #if num_clbits <= 6:
    #    print(ideal_result_dict)
    #else:
    #    print('result too long, skip')
        
    return ideal_result_dict

In [ ]:
def default_cost_return_fid(circ, layouts, backend, t=None):
    """The default mapomatic cost function that returns the total
    error rate over all the layouts for the gates in the given circuit
    Parameters:
        circ (QuantumCircuit): circuit of interest
        layouts (list of lists): List of specified layouts
        backend (IBMQBackend): An IBM Quantum backend instance
    Returns:
        list: Tuples of layout and error
    """
    out = []
    # Make a single layout nested
    if t:
        props = backend.properties(datetime=t)
    else:
        props = backend.properties()
        
    for layout in layouts:
        error = 0
        fid = 1
        for item in circ._data:
            if item[0].name == 'cx':
                q0 = circ.find_bit(item[1][0]).index
                q1 = circ.find_bit(item[1][1]).index
                fid *= (1-props.gate_error('cx', [layout[q0],
                                                  layout[q1]]))

            elif item[0].name in ['sx', 'x']:
                q0 = circ.find_bit(item[1][0]).index
                fid *= 1-props.gate_error(item[0].name, layout[q0])

            elif item[0].name == 'measure':
                q0 = circ.find_bit(item[1][0]).index
                fid *= 1-props.readout_error(layout[q0])
        error = 1-fid
        #out.append((layout, error))
        out.append((layout, fid))
    return out

In [ ]:
title = 'basis_trotter_n4'

with open('circuits_small/'+title+'.qasm', 'r') as f:
    qasm_str = f.read()
circuit = QuantumCircuit.from_qasm_str(qasm_str)

In [ ]:
circuit.draw(scale = 0.5)

In [ ]:
t = datetime(year=2022, month=11, day=15, hour=14)

backend_name = 'ibm_nairobi'
backend = provider.get_backend(backend_name)
backend.properties(datetime=t).last_update_date

In [ ]:
noise_model = NoiseModel.from_backend_properties(backend.properties(datetime=t))

# generate a simulator that mimics the real quantum system with the latest calibration results
#backend_sim = AerSimulator.from_backend(backend)

# generate a simulator that mimics the real quantum system with device properties
backend_sim = AerSimulator(noise_model=noise_model)

In [ ]:
trans_qc = transpile(circuit, backend, optimization_level=0)
print('# of CXs =', trans_qc.count_ops()['cx'])
display(trans_qc.draw(scale = 0.5))

# show qubit layout on the device
print('physical qubit layout:')
display(plot_gate_map(backend))
print('virtual qubit layout:')
display(plot_circuit_layout(trans_qc, backend, view='virtual'))

In [ ]:
shots = 1024
seed = None
layout = [[0,1,2,3]]

In [ ]:
mm_fid = default_cost_return_fid(mm.deflate_circuit(trans_qc), layout, backend)
mm_fid = mm_fid[0][1]
print(mm_fid)

In [ ]:
sim_result_dict = []
job_list = []
r2_list = []
num_clbits = trans_qc.num_clbits

num_tests = 50

for i in tqdm(list(range(num_tests))):
    sim_result_dict.append(run_ideal_simulation(trans_qc, shots, seed))
    job_list.append(backend_sim.run(trans_qc, shots=shots, seed_simulator=seed))
    
    temp_results = job_list[i].result()
    temp_result_dict = {}
        
    # build full result dict
    for j in range(pow(2, num_clbits)):
        bin_str = format(j, str('0>' + str(num_clbits) + 'b'))
        if bin_str not in temp_results.get_counts():
            #print(bin_str + ': probability = 0')
            temp_result_dict[bin_str] = 0
        else:
            temp_result_dict[bin_str] = temp_results.get_counts()[bin_str]
    
    r2_list.append(r2_score(list(sim_result_dict[i].values()), list(temp_result_dict.values())))
    
print(r2_list)
print(np.mean(r2_list))

In [ ]:
t = datetime(year=2022, month=11, day=18, hour=14)

noise_model = NoiseModel.from_backend_properties(backend.properties(datetime=t))
# generate a simulator that mimics the real quantum system with device properties
backend_sim = AerSimulator(noise_model=noise_model)

mm_fid_new = default_cost_return_fid(mm.deflate_circuit(trans_qc), layout, backend, t)
mm_fid_new = mm_fid_new[0][1]
print(mm_fid_new)

In [ ]:
sim_result_dict = []
job_list = []
r2_list_new = []
num_clbits = trans_qc.num_clbits

num_tests = 50

for i in tqdm(list(range(num_tests))):
    sim_result_dict.append(run_ideal_simulation(trans_qc, shots, seed))
    job_list.append(backend_sim.run(trans_qc, shots=shots, seed_simulator=seed))
    
    temp_results = job_list[i].result()
    temp_result_dict = {}
        
    # build full result dict
    for j in range(pow(2, num_clbits)):
        bin_str = format(j, str('0>' + str(num_clbits) + 'b'))
        if bin_str not in temp_results.get_counts():
            #print(bin_str + ': probability = 0')
            temp_result_dict[bin_str] = 0
        else:
            temp_result_dict[bin_str] = temp_results.get_counts()[bin_str]
    
    r2_list_new.append(r2_score(list(sim_result_dict[i].values()), list(temp_result_dict.values())))
    
print(r2_list_new)
print(np.mean(r2_list_new))

In [ ]:
title = 'basis_trotter_n4'

mm_fid = 0.000000337261284705294
q_fid = 0.07388155
r2_list = [0.06788533528645835, 0.05338541666666663, 0.056699625651041696, 0.05120442708333328, 0.08217569986979167, 0.055544026692708304, 0.1048868815104167, 0.0676554361979167, 0.033978271484375044, 0.07121988932291667, 0.035164388020833326, 0.04123942057291663, 0.04125366210937498, 0.06353352864583328, 0.05787556966145835, 0.049407958984375, 0.07328491210937504, 0.06786092122395837, 0.05643717447916663, 0.022580973307291696, 0.05921427408854163, 0.05817667643229163, 0.050687662760416696, 0.06954142252604167, 0.05236206054687498, 0.0795613606770833, 0.05861409505208337, 0.04925537109375, 0.0639668782552083, 0.05540364583333335, 0.095550537109375, 0.04543457031249998, 0.06560872395833328, 0.07370808919270833, 0.05219930013020835, 0.052834065755208326, 0.06748453776041663, 0.0582275390625, 0.052207438151041674, 0.012780761718749978, 0.10778198242187498, 0.06377766927083328, 0.06095987955729165, 0.07109171549479165, 0.053794352213541674, 0.059104410807291674, 0.06428426106770835, 0.06142781575520828, 0.06294759114583337, 0.06114095052083335]

mm_fid_new = 0.0000000157758337837269
q_fid_new = 0.056630462
r2_list_new = [0.04022827148437502, 0.060249837239583326, 0.054534912109375, 0.0912190755208333, 0.03660685221354165, 0.07555135091145837, 0.08918457031250004, 0.097320556640625, 0.10091349283854167, 0.05730183919270837, 0.10301310221354165, 0.08610229492187504, 0.07201131184895837, 0.07270507812500004, 0.08164469401041663, 0.10155436197916667, 0.07913818359375002, 0.05134073893229163, 0.06507161458333333, 0.08481852213541663, 0.05936279296875002, 0.06995035807291672, 0.03802083333333328, 0.056628417968749956, 0.07078653971354165, 0.08817545572916663, 0.018713378906250022, 0.10524291992187496, 0.09204915364583333, 0.09450073242187496, 0.070465087890625, 0.07806803385416672, 0.043892415364583326, 0.04127604166666665, 0.07667846679687496, 0.08020833333333333, 0.09329833984375002, 0.08191731770833333, 0.07077636718750002, 0.08122762044270837, 0.058809407552083304, 0.06105957031249998, 0.049582926432291674, 0.05805867513020835, 0.08821411132812496, 0.07434488932291672, 0.0729654947916667, 0.11236572265625, 0.10216267903645837, 0.06349283854166665]


In [ ]:
fig, ax = plt.subplots(figsize=(3,3))
ax.grid(color="dimgray", linestyle='dotted', linewidth=0.5)
#ax.set(xlabel='job index', ylabel='circuit fidelity', title='noisy simulation')
ax.set(title=title)
ax.set(ylim=(0.0, 1), yticks=[0, 0.5, 1], xticks=[0, 50, 100])
ax.xaxis.set_tick_params(labelsize='small')
ax.yaxis.set_tick_params(labelsize='small')

x = range(len(r2_list)*2)
avg = np.mean(r2_list)
ma = np.max(r2_list)
mi = np.min(r2_list)

ax.plot(x, r2_list+r2_list_new, color='dimgray', linewidth=3, label='ibm_nairobi = '+'{:.2f}'.format(np.mean(r2_list)) + ' → {:.2f}'.format(np.mean(r2_list_new)))
#ax.axhline(avg, color='red', linewidth=1, label='mean fidelity')
#ax.text(0, avg, "{:.2f}".format(avg), color="red", ha="right", va="top", fontsize='xx-large')
#ax.annotate("mean: {:.2f}".format(avg), xy=(0,avg-0.15), color='red', fontsize='x-large')
ax.axhline(q_fid, xmin=0, xmax=0.5, color='purple', linewidth=2, label='Q-fid prediction = '+'{:.2f}'.format(q_fid) + ' → {:.2f}'.format(q_fid_new))
ax.axhline(q_fid_new, xmin=0.5, xmax=1, color='purple', linewidth=2)
#for i in range(10):
#    ax.axhline(q_fid_new[i], xmin=0.5+i*0.05, xmax=0.55+i*0.05, color='purple', linewidth=2, label='q-fid estimation')

#ax.annotate("q-fid: {:.2f}".format(q_fid), xy=(50,q_fid+0.1), color='purple', fontsize='x-large', ha="right", va="top")
#ax.text(4, 0.1, # 0.7 or 0.1
#        "nairobi: {:.2f}".format(ma)+'\n'+"mean: {:.2f}".format(avg)+'\n'+"min:    {:.2f}".format(mi),
#        fontsize='small',
#        color="dimgray",
#        bbox=dict(facecolor='none', edgecolor='dimgray', boxstyle='round,pad=0.3'))
#ax.text(54, 0.1, # 0.7 or 0.1
#        "ibm_nairobi: \n           {:.2f}".format(ma)+'\n'+"Q-fid:   {:.2f}".format(avg)+'\n'+"mm:    {:.2f}".format(mi),
#        fontsize='medium',
#        color="dimgray",
#        bbox=dict(facecolor='none', edgecolor='dimgray', boxstyle='round,pad=0.3'))

plt.axvline(x = 50, color = 'b', linestyle='--', linewidth=1)

#fig.savefig(title+'_'+backend_name+'.png')

ax.axhline(mm_fid, xmin=0, xmax=0.5, color='orange', linewidth=1, label='mapomatic est. = '+'{:.2f}'.format(mm_fid) + ' → {:.2f}'.format(mm_fid_new))
ax.axhline(mm_fid_new, xmin=0.5, xmax=1, color='orange', linewidth=1)
#ax.annotate("mm: {:.2f}".format(mm_fid), xy=(50,mm_fid-0.02), color='dimgray', fontsize='medium', ha="right", va="top")
ax.legend(loc=0,fontsize='small')

print('RMSE q_fid_new:', np.sqrt(mean_squared_error(r2_list_new, [q_fid_new]*50)))
print('RMSE mm_fid_new:', np.sqrt(mean_squared_error(r2_list_new, [mm_fid_new]*50)))
plt.savefig('exp_3_img/'+title+'.png', dpi=500, bbox_inches = 'tight')